# 6.0 Interactive Inference Prototype
**Goal:** Build a quick, interactive UI within the notebook to validate the Multinomial-NB model's behavior across various inputs.

In [1]:
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output
from emoji_sentiment_analysis.config import MODELS_DIR
from emoji_sentiment_analysis.features import extract_emojis

# Load artifacts
model = joblib.load(MODELS_DIR / "sentiment_model.pkl")
vectorizer = joblib.load(MODELS_DIR / "tfidf_vectorizer.pkl")

In [ ]:
# UI Elements
text_input = widgets.Textarea(
    value='',
    placeholder='Type something with emojis...',
    description='Input:',
    layout=widgets.Layout(width='70%', height='100px')
)

button = widgets.Button(
    description='Predict Sentiment',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='search'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        raw_text = text_input.value
        if not raw_text.strip():
            print("Please enter some text.")
            return
            
        # 1. Feature Engineering (The emoji multiplier)
        emojis = extract_emojis(raw_text)
        processed_text = f"{raw_text} {emojis * 5}"
        
        # 2. Inference
        vec = vectorizer.transform([processed_text])
        pred = model.predict(vec)[0]
        prob = model.predict_proba(vec)[0] # Confidence scores
        
        # 3. Display Results
        label = "POSITIVE ✨" if pred == 1 else "NEGATIVE 😠"
        confidence = prob[1] if pred == 1 else prob[0]
        
        print(f"Engine Input: {processed_text}")
        print(f"Result: {label}")
        print(f"Confidence: {confidence:.2%}")

button.on_click(on_button_clicked)

# Render the UI
display(widgets.VBox([text_input, button, output]))